## Importing Libraries

In [1]:
!pip3 install transformers
!pip3 install datasets
!pip3 install sentencepiece
!pip3 install seqeval
!pip install transformers[torch]
!pip install accelerate
!pip install transformers
from datasets import load_dataset
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments,DataCollatorForTokenClassification
import accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=529dbc4e836cad179e3c9e31f12c3d9dc4c1ed14d0fa439f711764b0bf8e020c
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


2024-03-11 18:14:18.080787: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-11 18:14:18.080894: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-11 18:14:18.237452: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [1]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from sklearn.metrics import f1_score
import torch

## Prediction Function

In [2]:
def get_predictions( sentence, tokenizer, model ):
    # Let us first tokenize the sentence - split words into subwords
    tok_sentence = tokenizer(sentence, return_tensors='pt')
    with torch.no_grad():
        # we will send the tokenized sentence to the model to get predictions
        logits = model(**tok_sentence).logits.argmax(-1)

        # We will map the maximum predicted class id with the class label
        predicted_tokens_classes = [model.config.id2label[t.item()] for t in logits[0]]

        predicted_labels = []

        previous_token_id = 0
        # we need to assign the named entity label to the head word and not the following sub-words
        word_ids = tok_sentence.word_ids()
        for word_index in range(len(word_ids)):
            if word_ids[word_index] == None:
                previous_token_id = word_ids[word_index]
            elif word_ids[word_index] == previous_token_id:
                previous_token_id = word_ids[word_index]
            else:
                predicted_labels.append( predicted_tokens_classes[ word_index ] )
                previous_token_id = word_ids[word_index]
        return predicted_labels

## Loading Tokenizer and model from ai4Bharat Library

In [3]:
tokenizerNer = AutoTokenizer.from_pretrained("ai4bharat/IndicNER")
modelNer = AutoModelForTokenClassification.from_pretrained("ai4bharat/IndicNER")

tokenizer_config.json:   0%|          | 0.00/346 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/667M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


## Loading Naamapadam Dataset

In [4]:
from datasets import ClassLabel, load_dataset, load_metric, DownloadMode

lang='hi'

raw_datasets = load_dataset('ai4bharat/naamapadam', lang,)
train_data = raw_datasets['train']
test_data = raw_datasets['test']
val_data = raw_datasets['validation']

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset naamapadam_pr downloaded and prepared to /root/.cache/huggingface/datasets/ai4bharat___naamapadam_pr/hi/1.0.0/99b5ec77eabfaa3fbff510d8cf70d7c34519486cb7dbee99ede19474ddff9b20. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

## For mapping


In [5]:
padding = "max_length"
text_column_name = 'tokens'
label_column_name = 'ner_tags'

# Map NER labels to numerical values
label_mapping = {'O': 0, 'B-ORG': 1, 'I-ORG': 2, 'B-PER': 3, 'I-PER': 4, 'B-LOC': 5, 'I-LOC': 6}

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizerNer(
        examples[text_column_name],
        truncation=True,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(examples[label_column_name]):
        # print('=====')
        # print('{} {}'.format(i,label)) #ak
        word_ids = tokenized_inputs.word_ids(batch_index=i)

        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [8]:
tokenized_train_dataset =train_data.map(tokenize_and_align_labels,batched=True,load_from_cache_file=True,num_proc = 10)
tokenized_validation_dataset =val_data.map(tokenize_and_align_labels,batched=True,load_from_cache_file=True,num_proc = 10)
tokenized_test_dataset =test_data.map(tokenize_and_align_labels,batched=True,load_from_cache_file=True,num_proc = 10)

#0:   0%|          | 0/99 [00:00<?, ?ba/s]

#1:   0%|          | 0/99 [00:00<?, ?ba/s]

#2:   0%|          | 0/99 [00:00<?, ?ba/s]

#3:   0%|          | 0/99 [00:00<?, ?ba/s]

#4:   0%|          | 0/99 [00:00<?, ?ba/s]

...

#5:   0%|          | 0/1 [00:00<?, ?ba/s]

#7:   0%|          | 0/1 [00:00<?, ?ba/s]

#6:   0%|          | 0/1 [00:00<?, ?ba/s]

#9:   0%|          | 0/1 [00:00<?, ?ba/s]

#8:   0%|          | 0/1 [00:00<?, ?ba/s]

## Randomly Selecting dataset

In [9]:
token_train_1 = tokenized_train_dataset.shuffle(seed=42).select(range(40000))
token_valid_1 = tokenized_validation_dataset

In [10]:
modelNer=modelNer.to("cuda")
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments


2024-03-11 18:58:59.156595: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-11 18:58:59.156714: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-11 18:58:59.285533: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
!pip install transformers
!pip install accelerate -U

## Computing metrics which calculate various scores during training

In [13]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments,DataCollatorForTokenClassification
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

import numpy as np
def compute_metrics(p):
    predictions, labels = p.predictions, p.label_ids
    predictions = np.argmax(predictions, axis=2).flatten()  # Convert probabilities to class labels
    labels = labels.flatten()
    
    mask = (labels != -100)
    predictions = predictions[mask]
    labels = labels[mask]

    # Calculate macro F1 score
    f1 = f1_score(labels, predictions, average="macro")

    # Calculate precision, recall, and accuracy with zero_division parameter
    precision = precision_score(labels, predictions, average="macro", zero_division=1)
    recall = recall_score(labels, predictions, average="macro")
    accuracy = accuracy_score(labels, predictions)

    return {
        "macro_f1": f1,
        "precision": precision,
        "recall": recall,
        "accuracy": accuracy
    }

## Training Model

In [14]:
data_coll = DataCollatorForTokenClassification(tokenizerNer)
training_args = TrainingArguments(
    output_dir="./indic_ner_fine_tuned",
    num_train_epochs=4,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=16,
    logging_dir="./indic_ner_logs",
    evaluation_strategy="steps",  # Evaluate every epoch
    eval_steps=500,  # Adjust based on your training steps
    save_total_limit=2,
    metric_for_best_model="macro_f1",  # Specify the metric for early stopping
)

trainer = Trainer(
    model=modelNer,
    args=training_args,
    train_dataset=token_train_1,
    eval_dataset=token_valid_1,
    compute_metrics=compute_metrics,
    data_collator = data_coll # Use the custom metrics function
)

trainNer = trainer.train()


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss,Macro F1,Precision,Recall,Accuracy
500,0.319400,0.176625,0.828022,0.827426,0.829375,0.945573
1000,0.164600,0.176048,0.828849,0.826466,0.832976,0.945281
1500,0.164700,0.169545,0.828125,0.836612,0.821409,0.945987
2000,0.133500,0.175456,0.830315,0.837142,0.825730,0.946540
2500,0.123500,0.178879,0.826180,0.836656,0.817270,0.945041
3000,0.119300,0.176448,0.828714,0.831711,0.826254,0.946148
3500,0.115200,0.189987,0.826117,0.833363,0.819936,0.945422
4000,0.089200,0.200731,0.826036,0.839329,0.814284,0.945600
4500,0.094000,0.201071,0.825914,0.837421,0.816075,0.945488
5000,0.087300,0.200102,0.828025,0.837643,0.819606,0.945849


## Saved Model

In [15]:
trainer.save_model('./outputNer')

## Evaluating on test Dataset

In [16]:
trainer.evaluate(tokenized_test_dataset)

{'eval_loss': 0.17969352006912231,
 'eval_macro_f1': 0.8352556252594596,
 'eval_precision': 0.8352723095333197,
 'eval_recall': 0.8386104194562989,
 'eval_accuracy': 0.9522947770572563,
 'eval_runtime': 2.9324,
 'eval_samples_per_second': 295.66,
 'eval_steps_per_second': 18.756,
 'epoch': 4.0}